## Notes

- DB- Note on file names.  I slightly edited file names (I think to include a -) so the names  would be sufficiently uniform, for reading into dfs.  Let's ensure this is pushed correctly. 
- RE: CSV file inputs and date ranges.  I observe there is no 2022 autonomous vehicle disengagement report- driverless on the DMV website
- Similarily, there is something going on in the driverless mileage reports.
- The 2020-21-Autonomous-Mileage-Reports-CSV-driverless covers Jan 2020 - Nov 2021
- The 2021-22-Autonomous-Mileage-Reports-CSV-driverless covers Jan 2021 - Nov 2022, but only has 3 lines, for WERIDE CORP,
    - Which doesn't appear in the 2020-21 entry.  This is probably due to a delayed filing on their part, and staff did the ez thing.
- The 2022 Report starts in Dec 2021 - Nov 2022, and doesn't include WERIDE info (either by name or permit #)
- The 2023 Report covers Dec 2022- Nov 2023, and includes WERIDE, so there is no overlaping info.

- Re: The mileage reports.  I'm loading them in so the yeoman's work is done.  I don't want to botch joining them.
   - Some form of groupby + set_index, as per discord, might be the trick here. 
   - Alternatively, we can combine the permit # + VIN into a new single column, and this will enable some straightforward y/y granular tracking via joined dfs (m/m from Jan 2020 - Nov 2023)
- Finally, we should be on the lookout for permit #s that have company name changes, like the company got bought or something


# IMPORTS

In [77]:
import pandas as pd
import seaborn as sns 
import numpy as np
import plotly.express as px
from pprint import pprint
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
from bokeh.models import FactorRange, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook
from bokeh.palettes import Viridis
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# DATA

In [78]:
### Lists
## Years common across data
years = ["2020", "2021", "2022", "2023"]
years = sorted(list(years))

months = ["DEC-1", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JULY", "AUG", "SEP", "OCT", "NOV"]


### Mileage

In [79]:
# TODO: Cast DATE column as datetype for disengagements

## Read in mileage data
mile_dfs = []
for year in years:
    try:
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='utf-8')
    # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='latin-1')
    annual_mileage["Year"] = year
    annual_mileage.rename(columns={'ANNUAL TOTAL': 'Mileage Total'}, inplace=True)
    for i, month in enumerate(months, start=4):
        annual_mileage.columns.values[i] = month
    mile_dfs.append(annual_mileage)
    # INFO: each annual report has 22 cols

mileage_df = pd.concat(mile_dfs, ignore_index=True)
# mileage_df.set_index('VIN NUMBER', inplace=True)

File for year 2020 not found. Skipping...


First Time Filers - Mileage

In [80]:
mile_first_time_filer_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-first-time-filers.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-first-time-filers.csv", encoding='latin-1')
    mile_first_time_filer_dfs.append(annual_disengagements)
    
# Not combining due to join issue raised at top of document
#first_time_mileage_df = pd.concat(mile_first_time_filer, ignore_index=True)

#print(mile_first_time_filer_dfs[0].head())
print("# of objects in mile_driverless_dfs:  " + str(len(mile_first_time_filer_dfs)))

File for year 2023-24 not found. Skipping...
# of objects in mile_driverless_dfs:  3


Driverless Mileage

In [81]:
# DB - Using the same process I used above, for Autonomous-Mileage-Reports-CSV-driverless.csv & Autonomous-Mileage-Reports-CSV-first-time-filers.csv
mile_driverless_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-{str(int(year[3]) + 21)}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='latin-1')
    annual_disengagements["Year"] = year
    for i, month in enumerate(months, start=4):
        annual_disengagements.columns.values[i] = month
    mile_driverless_dfs.append(annual_disengagements)

for year in years:    
    try:
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='utf-8')
        # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
        # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV-driverless.csv", encoding='latin-1')
    annual_disengagements["Year"] = year
    for i, month in enumerate(months, start=4):
        annual_disengagements.columns.values[i] = month
    mile_driverless_dfs.append(annual_disengagements)

driverless_mileage_df = pd.concat(mile_driverless_dfs, ignore_index=True)
# TODO: deal with stupid double year file.

#print(mile_driverless_dfs[0].head())
print("# of objects in mile_driverless_dfs:  " + str(len(mile_driverless_dfs)))
driverless_mileage_df.head()


File for year 2022-23 not found. Skipping...
File for year 2023-24 not found. Skipping...
File for year 2020 not found. Skipping...
File for year 2021 not found. Skipping...
# of objects in mile_driverless_dfs:  4


,Manufacturer,Permit Number,VIN NUMBER,Annual Total of Disengagements,DEC-1,JAN,FEB,MAR,APR,MAY,...,JULY 2022,AUG 2022,SEP 2022,OCT 2022,NOV 2022,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,APOLLO AUTONOMOUS DRIVING USA LLC,AVDT006,3LN6L5LU2HR615153,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,APOLLO AUTONOMOUS DRIVING USA LLC,AVDT006,3LN6L2LU4GR629715,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,APOLLO AUTONOMOUS DRIVING USA LLC,AVDT006,2C4RC1N71LR2S2232,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CRUISE LLC,AVDT005,5G21A6P00L4100040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CRUISE LLC,AVDT005,5G21A6P01L4100029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Combined Mileage

In [82]:
# TODO Duplicate and modify code from combined disengagements here


### Disengagements

In [83]:
### Read in data
## Read in disengagements data
dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='utf-8')
    # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='latin-1')
    dis_dfs.append(annual_disengagements)
    
disengagement_df = pd.concat(dis_dfs, ignore_index=True)

#DB.  The unnamed column 10 only appears in the 2021 report, and the 71 entries are all for IMAGRY INC.,
# Reading "Perception" or "Planning"
# Dropped Column
disengagement_df = disengagement_df.drop('Unnamed: 9', axis=1)
#Apx 8k  empty rows.  Droped all rows with any nulls.  
disengagement_df.dropna(axis=0, how='any', inplace=True)

File for year 2020 not found. Skipping...


First Time Filers - Disengagements

In [84]:
# Adding in 1st time filer Disengagements.  Utilizing Kyle's code from above
first_time_dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-first-time-filers.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-first-time-filers.csv", encoding='latin-1')
    first_time_dis_dfs.append(annual_disengagements)
    
first_df = pd.concat(first_time_dis_dfs, ignore_index=True)
#Some 200ish null rows imported. Dropped any row with any missing data
first_df.dropna(axis=0, how='any', inplace=True)

File for year 2023-24 not found. Skipping...


Driverless Disengagements

In [85]:
# Driverless reports.  2022 is missing.  There are only 2 files here.  I've combined the above 2x for loops to snag them both, 
# since they have different naming conventions
driverless_dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='latin-1')
    driverless_dis_dfs.append(annual_disengagements)

for year in years:    
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='utf-8')
        # Allows the search to be broad
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
        # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='latin-1')
    driverless_dis_dfs.append(annual_disengagements)

autonomous_dis_df = pd.concat(driverless_dis_dfs, ignore_index=True)
# Dropping some 2000+ null rows.
autonomous_dis_df.dropna(axis=0, how='any', inplace=True)

File for year 2021-22 not found. Skipping...
File for year 2022-23 not found. Skipping...
File for year 2023-24 not found. Skipping...
File for year 2020 not found. Skipping...
File for year 2021 not found. Skipping...
File for year 2022 not found. Skipping...


Combined Disengagements

In [86]:
# 3x active DFs for the disengagement info are
# disengagement_df
# first_df
# autonomous_dis_df
# List of these objects to for loop the DATE cleanup.  
# IF we wanted to acccess the report source from the combined df, we could add a column to each, with the report source

# Initially had trouble by converting the dates to_datetime and then trying to drop the hour/min/sec.
# Resolved by first converting DATE to str, dropping everything after " " (since all times were separated from date by a space)
# And then subsequently converting the column to_datetime

# This works, but can't be re-run unless above code blocks that create the dfs are also re-run (I think).  
# Used print(i['DATE'].value_counts()) to confirm this came out correctly.  

disengagement_dfs_list = [disengagement_df, first_df, autonomous_dis_df]

for i in disengagement_dfs_list:
    i['DATE'] = i['DATE'].astype(str)
    i["DATE"] = i["DATE"].str.split(" ", expand=True)[0]
    i['DATE'] = pd.to_datetime(i['DATE'], errors='raise', dayfirst=True, format='mixed')

In [87]:
'''
for i in disengagement_dfs_list:
    #print(i['Permit Number'].value_counts())
    print(i.shape)
'''

# Operator-in-Vehicle = AVT Permit prefix
# NVO = AVDT Permit Prefix.
# I'm fairly certain splitting these into distinct columns for purposes of multi-indexed rows or columns is straightforward, 
# So am moving forward with data cleaning

"\nfor i in disengagement_dfs_list:\n    #print(i['Permit Number'].value_counts())\n    print(i.shape)\n"

In [88]:
# 3x disengagement df's combined together into one, single column for all permit info
combined_dis_df = pd.concat(disengagement_dfs_list, ignore_index=True)


# Data Cleaning and QA

Drop Columns

In [89]:
# What are "Unnamed" colums? Probably useless "reserved"
# This and below are from kyle, and deal with the initial mileage_df
unnamed_cols = ['Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21']
for check in unnamed_cols:
    non_null_count = mileage_df[check].notna().sum()
    print(f'Number of non values in {check}: {non_null_count}')

# Empty. Drop 'em
mileage_df = mileage_df.drop(columns=unnamed_cols)
mileage_df.columns


Number of non values in Unnamed: 17: 0
Number of non values in Unnamed: 18: 0
Number of non values in Unnamed: 19: 0
Number of non values in Unnamed: 20: 0
Number of non values in Unnamed: 21: 0


Index(['Manufacturer', 'Permit Number', 'VIN NUMBER',
       'Annual Total of Disengagements', 'DEC-1', 'JAN', 'FEB', 'MAR', 'APR',
       'MAY', 'JUN', 'JULY', 'AUG', 'SEP', 'OCT', 'NOV', 'Mileage Total',
       'Year'],
      dtype='object')

Set Types

In [90]:
### Mileage
print(f'Before: {mileage_df.dtypes}')
cast_as_float = [
       'Annual Total of Disengagements', 'DEC-1', 'JAN', 'FEB', 'MAR', 'APR',
       'MAY', 'JUN', 'JULY', 'AUG', 'SEP', 'OCT', 'NOV', 'Mileage Total',
       ]

for col in cast_as_float:
    mileage_df[col] = mileage_df[col].apply(lambda x: x.replace(',', '').replace(' ', '') if isinstance(x, str) else x)
    mileage_df[col] = mileage_df[col].astype(float)

# Assuming df is your DataFrame and 'Column_Name' is the name of the column you want to convert to represent years
mileage_df['Year'] = pd.to_datetime(mileage_df['Year']).dt.year
mileage_df["Manufacturer"] = mileage_df["Manufacturer"].astype(str)


print(f'After: {mileage_df.dtypes}')



Before: Manufacturer                       object
Permit Number                      object
VIN NUMBER                         object
Annual Total of Disengagements    float64
DEC-1                              object
JAN                                object
FEB                                object
MAR                                object
APR                                object
MAY                                object
JUN                                object
JULY                               object
AUG                                object
SEP                                object
OCT                                object
NOV                                object
Mileage Total                      object
Year                               object
dtype: object
After: Manufacturer                       object
Permit Number                      object
VIN NUMBER                         object
Annual Total of Disengagements    float64
DEC-1                             float64
JAN  

QA: Ensure sum of monthly miles still equals Annual totals after column shuffling

In [91]:
### QA the monthly vs sum values
# Calculate the sum of values in the month columns for each row
month_sums = mileage_df.iloc[:, 4:-2].sum(axis=1)  

# Calculate the absolute difference between the sum of month columns and the "Mileage Total" column
absolute_difference = abs(month_sums - mileage_df['Mileage Total'])

# Check if the absolute difference is less than or equal to 2.0
within_tolerance = absolute_difference <= 2.0
fishy = (~within_tolerance).sum()
print("Number of rows where months do not add up to within 1.0 of annual total:", fishy)

# Print rows where the absolute difference is not within 2.0
# print(mileage_df[~within_tolerance])
qa = mileage_df[~within_tolerance]
print(f'Rows where MileageTotal is null: {qa["Mileage Total"].isnull().sum()}')
qa.head()


# count all nulls 
print(mileage_df.isnull().sum())
mileage_df.dropna(subset=['Mileage Total'], inplace=True)
# Drops majority of other nulls
print(mileage_df.isnull().sum())


Number of rows where months do not add up to within 1.0 of annual total: 78
Rows where MileageTotal is null: 77
Manufacturer                       0
Permit Number                     30
VIN NUMBER                        30
Annual Total of Disengagements    30
DEC-1                             30
JAN                               30
FEB                               31
MAR                               31
APR                               31
MAY                               31
JUN                               32
JULY                              32
AUG                               33
SEP                               32
OCT                               33
NOV                               32
Mileage Total                     77
Year                               0
dtype: int64
Manufacturer                      0
Permit Number                     0
VIN NUMBER                        0
Annual Total of Disengagements    0
DEC-1                             0
JAN                          

Rename Columns

In [92]:
combined_dis_df.rename(columns = {"Manufacturer" : "OPERATOR",
                                  "DISENGAGEMENT\nLOCATION\n(Interstate, Freeway, Highway, Rural Road, Street, or Parking Facility)" : "DISENGAGEMENT LOCATION", 
                                  "DISENGAGEMENT INITIATED BY\n(AV System, Test Driver, Remote Operator, or Passenger)" : "INITIATED BY",
                                  "DRIVER PRESENT\n(Yes or No)" : "DRIVER PRESENT",
                                  "VEHICLE IS CAPABLE OF OPERATING WITHOUT A DRIVER\n(Yes or No)" : "NVO CAPABLE"}, inplace=True)


mileage_df.rename(columns = {"Manufacturer" : "OPERATOR"}, inplace=True)
combined_dis_df.head(10)



,OPERATOR,Permit Number,DATE,VIN NUMBER,NVO CAPABLE,DRIVER PRESENT,INITIATED BY,DISENGAGEMENT LOCATION,DESCRIPTION OF FACTS CAUSING DISENGAGEMENT
0,AIMOTIVE INC.,AVT036,2021-04-03,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,During an exit/merge the test vehicle was goin...
1,AIMOTIVE INC.,AVT036,2021-03-16,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,"Exit/Merge Lane departure, due to the car miss..."
2,AIMOTIVE INC.,AVT036,2021-03-16,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,"Exit/Merge Lane departure, due to the car miss..."
3,AIMOTIVE INC.,AVT036,2021-03-16,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,During an exit/merge the test vehicle was goin...
4,AIMOTIVE INC.,AVT036,2021-02-04,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,"The car should have stayed in the left lane, b..."
5,AIMOTIVE INC.,AVT036,2021-02-04,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,"Exit/Merge Lane departure, due to the car miss..."
6,AIMOTIVE INC.,AVT036,2021-04-13,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,During an exit/merge the test vehicle was goin...
7,AIMOTIVE INC.,AVT036,2021-04-14,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,During an exit/merge the test vehicle was goin...
8,AIMOTIVE INC.,AVT036,2021-04-16,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,During an exit/merge the test vehicle was goin...
9,AIMOTIVE INC.,AVT036,2021-04-16,JTDKN3DU9A0059509,No,Yes,Test Driver,Freeway,"The car should have stayed in the left lane, b..."


Reduce Duplicates

In [93]:

# DEDUPLICATOR
def polish(df, column):
   df[column]=df[column].str.upper() # Capitalize everything
   df[column] = df[column].str.replace("[.,]", "", regex=True) # Remove periods and commas 
   df[column] = df[column].str.replace(r'\b(LLC|INC)\b', '', regex=True)  # Remove "LLC" and "INC"
   df[column]=df[column].str.rstrip() # Remove trailing whitespace

   return df 

# Print statements are for verification and team review. Might leave them to show class "This is the mess of data we had to clean up" - discuss
print(f'Locations: {combined_dis_df["DISENGAGEMENT LOCATION"].nunique()} - {sorted(combined_dis_df["DISENGAGEMENT LOCATION"].unique())}')
print(f'Operators: {combined_dis_df["OPERATOR"].nunique()} - {sorted(combined_dis_df["OPERATOR"].unique())}')
print()

dis_columns_to_clean = ["DISENGAGEMENT LOCATION", "OPERATOR", "INITIATED BY"]

# Deuplicate Disengagements dataframe
for i in dis_columns_to_clean:
   combined_dis_df = polish(combined_dis_df, i)
# Set VIN as Index
#combined_dis_df.set_index("VIN NUMBER", inplace=True)


print(f'Cleaned locations: {combined_dis_df["DISENGAGEMENT LOCATION"].nunique()} - {sorted(combined_dis_df["DISENGAGEMENT LOCATION"].unique())}')
print(f'Cleaned operators: {combined_dis_df["OPERATOR"].nunique()} - {sorted(combined_dis_df["OPERATOR"].unique())}')
print()

# Deduplicate Mileage
print("Original number of operators: ", mileage_df["OPERATOR"].nunique())

miles_columns_to_clean = ["OPERATOR"]

# Deuplicate Operators
for i in miles_columns_to_clean:
   mileage_df = polish(mileage_df, i)

# TODO: Mop up the last operators name alignment
operator_name_dic = {"APOLLO AUTONOMOUS DRIVING USA" : "APOLLO",
                     "AUTOX TECHNOLOGIES" : "AUTOX",
                     'MERCEDES-BENZ RESEARCH & DEVELOPMENT NORTH AMERICA' : 'MERCEDES-BENZ',
                     'NISSAN NORTH AMERICA  DBA ALLIANCE INNOVATION LAB' : 'NISSAN',
                     'NVIDIA CORPORATION' : 'NVIDIA'
}

mileage_df["OPERATOR"] = mileage_df["OPERATOR"].replace(operator_name_dic)

print("Actual number of unique operators: ", mileage_df["OPERATOR"].nunique())

operators = sorted(mileage_df["OPERATOR"].unique())
for value in operators:
    pprint(value)




Locations: 15 - ['Express Way', 'Freeway', 'Freeway ', 'HIGHWAY', 'HIghway', 'Highway', 'Interstate', 'Parking Facility', 'Parking facility', 'Rural Road', 'STREET', 'Street', 'Urban', 'freeway', 'street']
Operators: 51 - ['AIMOTIVE INC.', 'APOLLO AUTONOMOUS DRIVING USA LLC', 'APPLE INC.', 'ARGO AI, LLC', 'AURORA OPERATIONS, INC.', 'AUTOX TECHNOLOGIES, INC', 'Apollo', 'BOSCH', 'CRUISE LLC', 'DEEPROUTE.AI', 'DIDI RESEARCH AMERICA LLC', 'DIDI RESEARCH AMERICA, LLC', 'Didi Research America', 'EASYMILE', 'GATIK AI INC', 'GATIK AI INC.', 'GHOST AUTONOMY INC', 'Gatik AI Inc', 'IMAGRY INC', 'IMAGRY INC ', 'IMAGRY INC.', 'INTEL CORPORATION', 'LYFT', 'MERCEDES-BENZ RESEARCH & DEVELOPMENT NORTH AMERICA, INC.', 'MOTIONAL AD, INC.', 'Motional AD, Inc.', 'NISSAN NORTH AMERICA, INC DBA ALLIANCE INNOVATION LAB', 'NURO, INC', 'NVIDIA', 'NVIDIA CORPORATION ', 'Nissan North America', 'Nuro Inc', 'PONY.AI, INC.', 'QCRAFT INC.', 'QUALCOMM TECHNOLOGIES, INC.', 'TOYOTA RESEARCH INSTITUTE', 'TOYOTA RESEARCH 

### Find 'Zippy'

In [ ]:
miles_by_vin = mileage_df.groupby("VIN NUMBER")["Mileage Total"].sum().sort_values(ascending=False)
miles_by_vin.head()
zippy = miles_by_vin.idxmax()
print(f'Zippy VIN is {zippy}')

print(f'Zippy is a {mileage_df.loc[mileage_df["VIN NUMBER"] == zippy, "OPERATOR"].unique()[0]} vehicle.')

### GRAPHS

### Big Picture

In [ ]:
# Create DF without zero mile cars
nonzero_mileage = mileage_df[(mileage_df["Mileage Total"] > 0)]

In [ ]:
#### Histogram: miles driven / VIN ######
fig = px.histogram(nonzero_mileage,
             x='Mileage Total', color="OPERATOR", title="Autonomous Mileage by Vehicle (2021-2023)", nbins=50)
fig.update_layout(xaxis_title='Vehicle Mileage', yaxis_title='Number of Vehicles', height=600, width=1000)
fig.show()

#### Histogram: miles driven / Operator ######
manufacturer_miles_sum = nonzero_mileage.groupby('OPERATOR')['Mileage Total'].sum().reset_index()

fig = px.histogram(manufacturer_miles_sum, x='Mileage Total', title='Total Miles Driven Per Operator (2021-2023)', hover_name="OPERATOR",
                   labels={'Mileage Total': 'Total Miles Driven'},  nbins=50, color="OPERATOR",
                    color_discrete_sequence=px.colors.sequential.Viridis)
fig.update_layout(yaxis_title='Operators', height=600, width=1000)
fig.show()


#### Total Miles driven per year 


In [ ]:
output_notebook()

In [ ]:
# Aggregate the sum of Mileage Total by Year and OPERATOR
agg_df = mileage_df.groupby(['OPERATOR', 'Year'])['Mileage Total'].sum().reset_index()
    
# Filter to include only data for the top 10 operators\n",
top_7_operators = mileage_df.groupby('OPERATOR')['Mileage Total'].sum().nlargest(7).index
top_miles = agg_df[agg_df['OPERATOR'].isin(top_7_operators)]
manufacturer_order = agg_df.groupby('OPERATOR')['Mileage Total'].sum().sort_values(ascending=False).index
line_go_up = mileage_df.groupby(['Year'])['Mileage Total'].sum().reset_index()

In [ ]:
# Create subplots to allow for a shared x-axis
fig = make_subplots(specs=[[{"secondary_y": False}]])

# Define colors using Viridis color scale
colors = px.colors.sequential.Viridis[:len(top_miles['OPERATOR'].unique())]

# Add bar chart for each operator
operators = top_miles['OPERATOR'].unique()
for i, operator in enumerate(operators):
    df_filtered = top_miles[top_miles['OPERATOR'] == operator]
    fig.add_trace(go.Bar(x=df_filtered['Year'], y=df_filtered['Mileage Total'], name=f"{operator}", text=operator,
                         marker_color=colors[i]), secondary_y=False)

# Add line graph
fig.add_trace(go.Scatter(x=line_go_up['Year'], y=line_go_up['Mileage Total'], mode='lines+markers', name='Industry Yearly Total',
                         marker_color='black'), secondary_y=False)

# Update axes and layout
fig.update_layout(title_text="Mileage by Year and Operator", height=600, width=1200)
fig.update_xaxes(title_text="Year", tickmode='linear', dtick=1)
fig.update_yaxes(title_text="Mileage Total", secondary_y=False, showgrid=True)

fig.write_image("Images/annual_mileage.jpg")
fig.show()

### Disengagements across industry

In [ ]:
combined_dis_df.head()

In [76]:
combined_dis_df["OPERATOR"] = combined_dis_df["OPERATOR"].replace(operator_name_dic)



combined_dis_df["Year"] = combined_dis_df["DATE"].dt.year
where_dis = combined_dis_df.groupby(["DISENGAGEMENT LOCATION"]).size().reset_index(name='Count')

fig = px.bar(where_dis.sort_values("Count", ascending=False),
             x="DISENGAGEMENT LOCATION",
             y="Count",
             #color="OPERATOR",
             color_discrete_sequence= px.colors.sequential.Viridis)
fig.update_layout(height=600, width=700, title="Location of Disengagements")
fig.write_image("Images/disengagement_locs.jpg")
fig.show()

who_dis = combined_dis_df.groupby(["INITIATED BY"]).size().reset_index(name='Count')
fig = px.bar(who_dis.sort_values("Count", ascending=False),
             x="INITIATED BY",
             y="Count",
             #color="OPERATOR",
             color_discrete_sequence= px.colors.sequential.Viridis)
fig.update_layout(height=600, width=700, title="Disengagement Triggered By:")
fig.write_image("Images/disengagement_why.jpg")
fig.show()

In [ ]:
#### DISENGAGEMENTS PER THOUSAND MILES DRIVEN
# Makes most sense to show by year on a per-company basis. 

# Group by manufacturer - get VIN count, total disengagments and total miles
dptm = nonzero_mileage.groupby(['OPERATOR', 'Year']).agg({'Mileage Total': 'sum',
                                                              'Annual Total of Disengagements': 'sum',
                                                              'VIN NUMBER': 'nunique'}).reset_index()
# Calculate DPTM
dptm["Disengagements/Thousand Miles"] = (dptm['Annual Total of Disengagements'] / (dptm['Mileage Total'] / 1000))

# Lowest DPTM Operators
smoothest_operators = dptm.groupby('OPERATOR')['Disengagements/Thousand Miles'].sum().nsmallest(10).index
# Simple Yearly Aggregate of DPTM per Operator
agg_2 = dptm.groupby(['OPERATOR', 'Year'])['Disengagements/Thousand Miles'].sum().reset_index()
# agg_2 filtered by smoothest operators (best 10)
lowest_disengage_rate = agg_2[agg_2['OPERATOR'].isin(smoothest_operators)]

# DPTM df filtered down to 10 best @ DPTM
dptm_filtered_10 = dptm[dptm['OPERATOR'].isin(smoothest_operators)]

dptm_total = nonzero_mileage.groupby(['OPERATOR']).agg({'Mileage Total': 'sum',
                                                              'Annual Total of Disengagements': 'sum'}).reset_index()
dptm_total["Disengagements/Thousand Miles"] = (dptm_total['Annual Total of Disengagements'] / (dptm_total['Mileage Total'] / 1000))

dptm.head(30)

### Graphing RATE of dsengagements against total miles


In [ ]:
# Showing each operator each year 
fig = px.scatter(dptm, x="Mileage Total", y="Disengagements/Thousand Miles", color="OPERATOR",
                 color_discrete_sequence=px.colors.sequential.Viridis)
fig.update_layout(title='Operator Yearly Disengagements Per Thousand Miles v. Total Mileage - All Operators', height=600, width=1200)
fig.write_image("Images/disengagements_per_thousand_miles_vs_miles_driven.jpg")
fig.show()


Causes of Disengagement

In [ ]:
# Word cloud of Disengagement descriptions
text = combined_dis_df["DESCRIPTION OF FACTS CAUSING DISENGAGEMENT"].values 

coloring = np.array(Image.open('Images/av2.jpg'))
stop_words = ["vehicle", "/n", "Non", "road"] + list(STOPWORDS)
wordcloud = WordCloud(stopwords = stop_words,  max_font_size=500, random_state=3,mask=coloring).generate(str(text))

image_colors = ImageColorGenerator(coloring)

plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear")
plt.axis("off")
plt.figure()
wordcloud.to_file("Images/disengagements_wc.jpg")


## COLLSIONS !

In [ ]:
# Import time and place collision data
coll_time_place_df = pd.read_csv('data/collisions/dataframes/locations/crash_locations.csv')

# Clean it:
coll_columns_to_clean = ["Operator","Business"]
for i in coll_columns_to_clean:
   coll_time_place_df = polish(coll_time_place_df, i)

unique_operator = coll_time_place_df["Operator"].unique()
print(unique_operator)

coll_time_place_df.head()


Collision time of day histogram

In [ ]:
# temprorary column to fix binning issues
coll_time_place_df['Hour_of_day'] = pd.to_datetime(coll_time_place_df['Time_of_accident_24hr']).dt.strftime('%H')

# Plot a histogram with Plotly Express
fig = px.histogram(coll_time_place_df.sort_values('Hour_of_day'), 
                   x='Hour_of_day',
                   title='Autonomy Collisions by Time of Day',
                   nbins=24,  # One bin for each hour
                   labels={'Hour_of_day': 'Hour of Day', 'count': 'Number of Collisions'},
                   color='Business',  # Color by operator
                   color_discrete_sequence=px.colors.sequential.Viridis
                  )

fig.update_layout(xaxis_title='Hour of Day', yaxis_title='Number of Collisions', height=600, width=1200)
fig.write_image('Images/collisions_by_hour.jpg')
fig.show()

#drop temp column
coll_time_place_df.drop(columns=(["Hour_of_day"]), inplace=True)


Collision Day of Year histogram

In [ ]:
# Convert 'Date_of_Accident' to datetime
coll_time_place_df['Date_of_Accident'] = pd.to_datetime(coll_time_place_df['Date_of_Accident'])
# Get mm-DD pair
coll_time_place_df['Month'] = coll_time_place_df['Date_of_Accident'].dt.month
coll_time_place_df.head()

In [ ]:

# Group by month and count the number of collisions
collisions_by_month = coll_time_place_df.groupby('Month').size().reset_index(name='Count')

# Rename month numbers to month names for better visualization
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
               7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
collisions_by_month['Month'] = collisions_by_month['Month'].map(month_names)

# Plot histogram
fig = px.bar(collisions_by_month, x='Month', y='Count', 
             title="Autonomous Collisions by Month", 
             labels={'Month': 'Month', 'Count': 'Number of Collisions'},
             color='Month',  # Color by operator
             color_discrete_sequence=[px.colors.sequential.Viridis[1]]*len(collisions_by_month))

# Update layout
fig.update_layout(xaxis_title='Month', yaxis_title='Number of Collisions', showlegend=False, height=600, width=900)
fig.write_image("Images/collisions_by_month.jpg")
fig.show()

Collisions description wordcloud

In [ ]:
collision_description = pd.read_csv('data/collisions/dataframes/descriptions/descriptions.csv')

description = collision_description["Description"].values 

coloring = np.array(Image.open('Images/av2.jpg'))
stop_words = ["/n"] + list(STOPWORDS)
wordcloud = WordCloud(stopwords = stop_words,  max_font_size=100, random_state=4,mask=coloring).generate(str(description))

image_colors = ImageColorGenerator(coloring)

plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear")
plt.axis("off")
plt.figure()




Join collisions and Disengagements (per thousand miles)

In [ ]:
print(dptm.index)
print(dptm.columns)
coll_time_place_df.drop(columns=(['location', 'city', 'county', 'state', 'zip']), inplace=True)
print(coll_time_place_df.columns)


# Waymo

In [ ]:
# miles per month


In [ ]:
# Disengagements by year by type
# Make FOCUS dataframe
focus_company = "WAYMO"
focus_dis_df = combined_dis_df[combined_dis_df['OPERATOR'] == focus_company]
# Year column
focus_dis_df['Year'] = focus_dis_df['DATE'].dt.year

causes_dict = {'Disengage for a recklessly behaving road user' : 'Other driver',
 'Disengage for unwanted maneuver of the vehicle that was undesirable under the circumstances' : 'Motion Planning',
 "Disengage for a perception discrepancy for which a component of the vehicle's perception system failed to detect an object correctly" : "Perception",
 "Disengage for a hardware discrepancy for which our vehicle's diagnostics received a message indicating a potential performance issue with a hardware component of the self-driving system or a component of the base vehicle" : "Diagnositc: Hardware",
 "Disengage for a software discrepancy for which our vehicle's diagnostics received a message indicating a potential performance issue with a software component" : "Diagnositc: Software",
 'Disengage for incorrect behavior prediction of other traffic participants': "Prediciton"}

focus_dis_df["DESCRIPTION OF FACTS CAUSING DISENGAGEMENT"].replace(causes_dict,inplace=True)
 
grouped_focus = focus_dis_df.groupby(['DESCRIPTION OF FACTS CAUSING DISENGAGEMENT', 'Year']).size().reset_index(name='Count')


fig = px.bar(grouped_focus, x='Year', y='Count', color='DESCRIPTION OF FACTS CAUSING DISENGAGEMENT',
             title='WAYMO Disengagement Categories',
             labels={'DESCRIPTION OF FACTS CAUSING DISENGAGEMENT': 'Description of Disengagement Facts',
                     'Count': 'Count of Disengagement Facts'},            
             color_discrete_sequence=px.colors.sequential.Viridis,
             barmode='group')
fig.update_layout(height = 600, width = 1000)
fig.write_image("Images/Waymo_disengagements.jpg")
fig.show()


1. The above disengagement count seems way too low.
2. Could maybe group by the first clause before the ";"

# ZIPPY

In [ ]:
# graph of miles over time?
# Combined graph - line: miles, columns, disengagements?

In [ ]:
# sunburst of mileage by month? 
# what sotry? weather vehicles can/can't run in
